In [1]:
from azureml.core import Workspace

In [2]:
subscription_id = "40bffbcc-578f-4e44-bd6d-972552eb6513" # The ID of the Azure Subscription
resource_group = "gestaltml" # Name of a logical resource group
workspace_name = "fastai2" # The name of the workspace to look for or to create
workspace_region = 'East US' # Location of the workspace
#experiment_name = 'breastcancer'
score_script = 'score_and_track.py'
modelupload_name = 'breastcancerdetect'
service_name = 'breastcancerdetect'

In [3]:
#Create Workspace if needed
# ws = Workspace.create(name=workspace_name,
#                subscription_id=subscription_id,
#                resource_group=resource_group,
#                create_resource_group=True,
#                location=workspace_region,
#                )

In [4]:

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

Manually Upload & Register Model in Azure ML Workspace (or use register script below)

In [5]:
from azureml.core.model import Model

#OPen existin model already registered
model = Model(ws, modelupload_name)

# #Register model generated in Azure
# model = Model.register(model_path="export.pkl",
#                           model_name=modelupload_name,
#                           tags={'data': 'Babies', 'method':'transfer learning','framework':'pytorch'},
#                           #tags={'data': 'breastcancer',  'method':'transfer learning', 'framework':'pytorch'},
#                           description='Image classficiation of Baby Vibes',
#                           workspace=ws)

In [6]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment
from azureml.core.model import InferenceConfig

myenv = CondaDependencies.create(pip_packages=['azureml-defaults', 'fastai==1.0.61','pillow==5.4.1',
                                               'torch', 'torchvision>=0.5.0'])


# myenv = CondaDependencies.create(pip_packages=['azureml-defaults~=1.13.0', 'fastcore==1.0.0', 
# 'fastprogress==1.0.0', 'fastscript==1.0.0', 'Pillow==5.4.1', 'requests', 'torch', 'torchvision>=0.5.0', 'fastai2==0.0.30', 'fastai==2.0.6'])
 
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
    
print(myenv.serialize_to_string())

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score_and_track.py",
                                   environment=myenv)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-defaults
  - fastai==1.0.61
  - pillow==5.4.1
  - torch
  - torchvision>=0.5.0
channels:
- anaconda
- conda-forge



In [7]:
from azureml.core.webservice import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException



# Remove any existing service under the same name.
try:
   Webservice(ws, service_name).delete()
except WebserviceException:
   pass
    
aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=4, 
                                               tags={'data': 'breastcancer',  'method':'transfer learning', 'framework':'pytorch'},
                                               description='')

In [8]:
#Open existing webservice
#service = Webservice(ws, service_name)


In [9]:

#Deploy new webservice
service = Model.deploy(workspace=ws, 
                           name=service_name, 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

Running............................................................................................................................................................................................................................................................................................................................................................
TimedOut
Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 7edcdb7c-440d-4c73-8f45-4e0d28a152c5
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice. \nPlease re

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: 7edcdb7c-440d-4c73-8f45-4e0d28a152c5
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice. \nPlease refer to https://aka.ms/debugimage#dockerlog for more information.",
  "details": [
    {
      "code": "AciDeploymentFailed",
      "message": "Your container application crashed. Please follow the steps to debug:\n1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the 

In [10]:
service.get_logs()

'2020-09-21T04:27:13,780779471+00:00 - gunicorn/run \n2020-09-21T04:27:13,780778571+00:00 - rsyslog/run \n2020-09-21T04:27:13,781785571+00:00 - nginx/run \n2020-09-21T04:27:13,786272268+00:00 - iot-server/run \n/usr/sbin/nginx: /azureml-envs/azureml_735c2df7e059c8781874479b32c473b8/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_735c2df7e059c8781874479b32c473b8/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_735c2df7e059c8781874479b32c473b8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_735c2df7e059c8781874479b32c473b8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_735c2df7e059c8781874479b32c473b8/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [ ]:
print(service.scoring_uri)

In [ ]:
service.delete()

In [ ]:
service.state